# **Informe:**
https://datastudio.google.com/reporting/b65ca3e6-99e4-437f-b358-3150cab9a8be


In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import statistics as st
!pip install gspread --upgrade
import gspread
import webbrowser
from time import sleep

#Google Drive:
from google.colab import drive
drive.mount('/content/drive')

ruta_excel = '/content/drive/My Drive/Royal Distribuidora/FACTURACION ROYAL.xlsx'

# Conocer la cantidad y nombre de las hojas
xl = pd.ExcelFile(ruta_excel)
hojas = len(xl.sheet_names)
nombres_hojas = xl.sheet_names

# Creo lista de df's
dataframes = []

for sheet in nombres_hojas:
  df = pd.read_excel(ruta_excel, sheet_name = sheet)
  
  # Quedarnos con solo las columnas importantes
  columnas_relevantes =['marca','descripcion','costo uni','cant','total costo','Precio Venta',
          'IIBB','Cargo Mercado Pago','Comision Plataforma','Flete','ML Total','Ganacia neta x uni',
          'Total Gan Neta','Medio Venta','Tipo de Venta (Flex o Correo)',
          'Prov Cliente','Fecha']
  columnas_iniciales = list(df.columns)
  columnas_final = [col for col in columnas_iniciales if col in columnas_relevantes]
  df = df[columnas_final]

  # Agrego el tipo y cantidad de gastos
  for i,v in enumerate(df['costo uni'].values):
    if v == 'Cantidad' or v == 'cantidad':
      indice = i
  tipo_gasto = pd.DataFrame(list(df['descripcion'][indice+1:-1])).dropna()
  cantidad_gasto = pd.DataFrame(list(df['costo uni'][indice+1:-1]))[0:len(tipo_gasto)]
  df['tipo_gasto'] = tipo_gasto
  df['cantidad_gasto'] = cantidad_gasto
  
  # Drop de las filas que no sean productos
  df.drop(df.index[len(df.marca.dropna()):len(df.marca)], axis=0, inplace=True)
  df['mes'] = sheet
  
  # Agrego el df final a la lista de df's
  dataframes.append(df)

# Unir df
df = pd.concat(dataframes)
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)

#Limpieza
df.fillna('', inplace=True)
df['Fecha'] = df['Fecha'].astype(str)
for col in df.columns.tolist():
  try:
    df[col] = [x.strip() for x in df[col].values]
  except:
    pass

#Columnas nuevas
df['marca_prod'] = df['marca'] + ' '+ df['descripcion']
df['marca_prod_resumen'] = [df['marca'][i] + ' ' + x.split(' ')[0] for i,x in enumerate(df['descripcion'].values)]

# Spreadsheet
gc = gspread.service_account(filename='/content/drive/My Drive/Royal Distribuidora/Analisis Royal/credenciales.json')

#Create if not exists
try:
    hoja_de_calculo = gc.open('ResultadosAnalisisRoyal')
    print('\n\nAbriendo excel existente.\n\n')
except:
    hoja_de_calculo = gc.create('ResultadosAnalisisRoyal')
    print('\n\nExcel no encontrado, creando uno...\n\n')

#Compartirla
mails = ['lipe.aguirre@gmail.com','navarro.maty@yahoo.com.ar','swinn998@gmail.com']
[hoja_de_calculo.share(mail, perm_type='user', role='writer') for mail in mails]
hoja_de_calculo = gc.open("ResultadosAnalisisRoyal")

#Llenarla de datos
worksheet = hoja_de_calculo.sheet1
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Abriendo excel existente.




{'spreadsheetId': '1USYs6ao3YkaB8JI-BysDiLKlDCNDqYgCB4f6b2QAMpA',
 'updatedRange': 'Sheet1!A1:V808',
 'updatedRows': 808,
 'updatedColumns': 22,
 'updatedCells': 17776}

# **Analisis Especificos**